In [ ]:
import torch
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from etils import epath
from custom_metrics import monai_metrics, segmentation_metrics

import codebase.codebase_settings as cbs
from codebase.projects.hecktor2022 import evaluation

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

<h3> Single class output </h3>

In [ ]:
_METRICS_CONFIG = {'name': 'dice', 'include_background': False, 'reduction': 'none',
                   'get_not_nans': True, 'num_classes': 2}
metrics = monai_metrics.get_segmentation_metrics(_METRICS_CONFIG)

In [ ]:
def prepare_subvolume_batch(batch):
    """Create features and label for each batch."""
    features = batch['input'].to(device)
    mask = torch.sum(batch['label'][:, 1:3, ...], dim=1)
    mask = mask[:, None, ...].to(device)
    return features, mask

In [ ]:
checkpoint_path = '/workspace/ro_codebase/preprocessor/images/test_data/processed_128x128/subvolume_32/experiments/hecktor_test/best_model.pth'
# checkpoint_path = '/workspace/data/hecktor2022/processed_128x128/subvolume_32/set1/experiments/hecktor_exp061323/generalized_focal_dice_best_model.pth'
checkpoint_path = '/workspace/data/hecktor2022/processed_128x128/subvolume_32/set1/experiments/hecktor_exp061323/gfd_1class_best_model.pth'
checkpoint = torch.load(checkpoint_path)
model = checkpoint['model']
# trainer.model.load_state_dict(checkpoint['model_state_dict'])
# trainer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
best_loss = checkpoint['best_loss']
print(best_loss)

In [ ]:
data_folder = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data'
# image = torch.Tensor(np.load(str(data_folder / 'processed_128x128/subvolume_32/train/images/CHUM-024_38__input.npy')))
# label = torch.Tensor(np.load(str(data_folder / 'processed_128x128/subvolume_32/train/labels/CHUM-024_38__label.npy')))
# image = torch.Tensor(np.load(str(data_folder / 'processed_128x128/subvolume_32/train/images/CHUM-024_25__input.npy')))
# label = torch.Tensor(np.load(str(data_folder / 'processed_128x128/subvolume_32/train/labels/CHUM-024_25__label.npy')))
image = torch.Tensor(np.load('/workspace/data/hecktor2022/processed_128x128/subvolume_32/valid/images/CHUV-008_239__input.npy'))
label = torch.Tensor(np.load('/workspace/data/hecktor2022/processed_128x128/subvolume_32/valid/labels/CHUV-008_239__label.npy'))
batch = {'input': image[None, ...], 'label': label[None, ...]}
features, targets = prepare_subvolume_batch(batch)

In [ ]:
# prediction = model(image[None].to(device))
print(features.shape, targets.shape)
prediction = model(features)
prediction = torch.sigmoid(prediction)
prediction = (prediction > 0.5).int()
print(prediction.shape)

In [ ]:
dice_value = metrics([prediction], [targets])
dice_value

In [ ]:
prediction = prediction[0].detach().cpu().numpy()
targets = targets[0].cpu().numpy()
inputs = features[0].cpu().numpy()
ct = inputs[0, ...]
pet = inputs[1, ...]
# prediction = image.numpy()
prediction.shape

In [ ]:
# all_imgs = [label.data[0, :, :, :].numpy(), label.data[1, :, :, :].numpy(), label.data[2, :, :, :].numpy(),]
# all_imgs = np.swapaxes(prediction, 1, 3)
all_imgs = np.array([np.swapaxes(pet, 0, 2), np.swapaxes(prediction[0], 0, 2), np.swapaxes(targets[0], 0, 2)])
px.imshow(
    all_imgs,
    # zmin=[0, 0, 0],
    # zmax=[2000, 2000, 2000],
    animation_frame=1,
    # binary_string=gray_scale,
    labels={'animation_frame': 'slice'},
    facet_col=0,
    color_continuous_scale='Gray',
    width=500*3, height=500
)

<h2> Cohort Evaluation </h2>

In [ ]:
checkpoint_path = '/workspace/ro_codebase/preprocessor/images/test_data/processed_128x128/subvolume_32/experiments/hecktor_test/best_model.pth'
# checkpoint_path = '/workspace/data/hecktor2022/processed_128x128/subvolume_32/set1/experiments/hecktor_exp061323/generalized_focal_dice_best_model.pth'
checkpoint_path = '/workspace/data/hecktor2022/processed_128x128/subvolume_32/set1/experiments/hecktor_exp061323/gfd_1class_best_model.pth'

data_path = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data' / 'processed_128x128' / 'subvolume_32' / 'train'
data_path = cbs.DATA_PATH / 'hecktor2022' / 'processed_128x128' / 'subvolume_32' / 'valid'

In [ ]:
evaluator = evaluation.EvaluationModule(epath.Path(checkpoint_path), epath.Path(data_path))

In [ ]:
results = evaluator.evaluate_cohort()

In [ ]:
cohort1 = results

In [ ]:
cohort2 = results

In [ ]:
cohort_both = np.concatenate((cohort1, cohort2))

In [ ]:
plt.hist(cohort_both, bins=100)

In [ ]:
print(len(cohort_both))
print(np.nanmean(cohort_both))

In [ ]:
np.mean(np.nan_to_num(cohort_both, nan=1))